In [1]:
from bs4 import BeautifulSoup as bs
import requests
import lxml
import pandas as pd

In [2]:
races = pd.read_csv('RacesTableau.csv')
races

,raceId,year,Grand Prix,Driver,Constructor,points
0,776,1957,Argentine Grand Prix,Juan Fangio,Maserati,8
1,776,1957,Argentine Grand Prix,Jean Behra,Maserati,6
2,776,1957,Argentine Grand Prix,Carlos Menditeguy,Maserati,4
3,776,1957,Argentine Grand Prix,Harry Schell,Maserati,3
4,776,1957,Argentine Grand Prix,Alfonso de Portago,Ferrari,1
...,...,...,...,...,...,...
23615,1074,2022,Bahrain Grand Prix,Nicholas Latifi,Williams,0
23616,1074,2022,Bahrain Grand Prix,Nico Hulkenberg,Aston Martin,0
23617,1074,2022,Bahrain Grand Prix,Sergio Perez,Red Bull,0
23618,1074,2022,Bahrain Grand Prix,Max Verstappen,Red Bull,0


In [3]:
raceid=94
year=1950
RaceResults = pd.DataFrame(columns = ['Raceid', 'Year','Grand Prix', 'Driver', 'Constructor', 'Points'])
while year!=1957:
    url = f'https://www.formula1.com/en/results.html/{year}/races/{raceid}/.+/race-result.html'
    response = requests.get(url)
    soup = bs(response.text,'lxml')
    grandprix = soup.select('h1.ResultsArchiveTitle')[0].text.strip()
    if grandprix[5:-23] == "":
        year = year+1
    else:
        gp = grandprix[5:-23]
        table = soup.find('table',class_='resultsarchive-table')
        for row in table.tbody.find_all('tr'):
            rows = row.find_all('td')
            driver=rows[3].text
            driver = driver[1:-5]
            constructor=rows[4].text
            points = rows[7].text
            RaceResults.loc[len(RaceResults.index)] = [raceid, year,gp,driver,constructor,points]
        raceid=raceid+1


In [4]:
RaceResults

,Raceid,Year,Grand Prix,Driver,Constructor,Points
0,94,1950,British Grand Prix,Nino\nFarina,Alfa Romeo,9
1,94,1950,British Grand Prix,Luigi\nFagioli,Alfa Romeo,6
2,94,1950,British Grand Prix,Reg \nParnell,Alfa Romeo,4
3,94,1950,British Grand Prix,Yves\nGiraud-Cabantous,Talbot-Lago,3
4,94,1950,British Grand Prix,Louis \nRosier,Talbot-Lago,2
...,...,...,...,...,...,...
1351,149,1956,Italian Grand Prix,Luigi \nVilloresi,Maserati,0
1352,149,1956,Italian Grand Prix,Jo\nBonnier,Maserati,0
1353,149,1956,Italian Grand Prix,Alfonso\nde Portago,Ferrari,0
1354,149,1956,Italian Grand Prix,Les \nLeston,Connaught Alta,0


In [22]:
races.rename({'raceId':'Raceid','year':'Year','points':'Points'},axis='columns',inplace=True)

In [15]:
RaceResults[['Driver']] = RaceResults[['Driver']].replace('\n',' ',regex=True) 

In [16]:
RaceResults

,Raceid,Year,Grand Prix,Driver,Constructor,Points
0,94,1950,British Grand Prix,Nino Farina,Alfa Romeo,9
1,94,1950,British Grand Prix,Luigi Fagioli,Alfa Romeo,6
2,94,1950,British Grand Prix,Reg Parnell,Alfa Romeo,4
3,94,1950,British Grand Prix,Yves Giraud-Cabantous,Talbot-Lago,3
4,94,1950,British Grand Prix,Louis Rosier,Talbot-Lago,2
...,...,...,...,...,...,...
1351,149,1956,Italian Grand Prix,Luigi Villoresi,Maserati,0
1352,149,1956,Italian Grand Prix,Jo Bonnier,Maserati,0
1353,149,1956,Italian Grand Prix,Alfonso de Portago,Ferrari,0
1354,149,1956,Italian Grand Prix,Les Leston,Connaught Alta,0


In [23]:
df = pd.concat([RaceResults, races])
df

,Raceid,Year,Grand Prix,Driver,Constructor,Points
0,94,1950,British Grand Prix,Nino Farina,Alfa Romeo,9
1,94,1950,British Grand Prix,Luigi Fagioli,Alfa Romeo,6
2,94,1950,British Grand Prix,Reg Parnell,Alfa Romeo,4
3,94,1950,British Grand Prix,Yves Giraud-Cabantous,Talbot-Lago,3
4,94,1950,British Grand Prix,Louis Rosier,Talbot-Lago,2
...,...,...,...,...,...,...
23615,1074,2022,Bahrain Grand Prix,Nicholas Latifi,Williams,0
23616,1074,2022,Bahrain Grand Prix,Nico Hulkenberg,Aston Martin,0
23617,1074,2022,Bahrain Grand Prix,Sergio Perez,Red Bull,0
23618,1074,2022,Bahrain Grand Prix,Max Verstappen,Red Bull,0


In [25]:
df.to_csv('AllSeasonRaces.csv')